In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random

In [5]:
screenshots_parent = './screenshots_dataset'
fen_parent = './fen_dataset/train'
output_parent = './generated_images/eval'

In [6]:
def is_white(img, threshold=200):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    white_pixels = np.sum(gray_img >= threshold)
    total_pixels = gray_img.shape[0] * gray_img.shape[1]
    
    if white_pixels / total_pixels >= 0.95:
        return True
    else:
        return False

def load_random_screenshots(parent_folder, num=1, target_size=(640, 640)):
    images = []
    subdirs = random.choices(os.listdir(parent_folder), k=num)

    for subdir in subdirs:
        file_path = os.path.join(parent_folder, subdir, "screenshot.png")
        
        img = cv2.imread(file_path)
        if is_white(img):
            continue
        img = cv2.resize(img, target_size)
        
        images.append(img)

    return images


def load_random_boards(parent_folder, num=1):
    images = []
    image_names = random.choices(os.listdir(parent_folder), k=num)
    
    for img_name in image_names:
        file_path = os.path.join(parent_folder, img_name)

        img = cv2.imread(file_path)
        
        images.append(img)

    return images


def generate_image(output_parent, num):
    screenshots = load_random_screenshots(screenshots_parent, num=num)
    boards = load_random_boards(fen_parent, num=len(screenshots))
    last_index = len(os.listdir(os.path.join(output_parent, "images")))

    for i in range(len(screenshots)):
        random_size = np.random.uniform(0.25, 0.75)
        resized_board = cv2.resize(boards[i], (0, 0), fx=random_size, fy=random_size)
        
        # Calculate metadata
        screenshot_height, screenshot_width, _ = screenshots[i].shape
        board_height, board_width, _ = resized_board.shape

        x_offset = board_width / (screenshot_width * 2)
        y_offset = board_height / (screenshot_height * 2)

        x = np.random.uniform(x_offset, 1 - x_offset)
        y = np.random.uniform(y_offset, 1 - y_offset)
        w = board_width / screenshot_width
        h = board_height / screenshot_height

        # Overlay resized board onto the screenshot
        x_pixel = int(x * screenshot_width)
        y_pixel = int(y * screenshot_height)

        x_start = max(x_pixel - board_width // 2, 0)
        y_start = max(y_pixel - board_height // 2, 0)
        x_end = min(x_pixel + board_width // 2, screenshot_width)
        y_end = min(y_pixel + board_height // 2, screenshot_height)
        screenshots[i][y_start:y_end, x_start:x_end] = resized_board[0:y_end - y_start, 0:x_end - x_start]

        output_path = os.path.join(output_parent, "images", f'image_{last_index + i}.jpg')
        metadata_path = os.path.join(output_parent, "labels", f'image_{last_index + i}.txt')

        # Save the image
        cv2.imwrite(output_path, screenshots[i])

        # Save the metadata
        with open(metadata_path, 'w') as f:
            f.write(f'0 {x} {y} {w} {h}')
    
    return screenshots


In [7]:
images = generate_image(output_parent, num=2000)
# plt.imshow(images[0])